In [ ]:
# %pip install -r requirements.txt
from IPython.display import clear_output
#clear_output()
from enum import Enum, auto
from dataclasses import dataclass, fields
import typing
from typing import Any, Tuple
import random
from pprint import pprint
import asyncio
import sys
import threading
import time

import ipywidgets as widgets

%load_ext mypy_ipython

In [ ]:
def weighted_if(weight, out1, out2):
    return out1 if random.random() < weight else out2

In [ ]:
@dataclass(frozen=True, eq=True)
class Allele:
    value: Any
    dominant: bool
    
    def mutate(self, other):
        mut = mutations.get((self, other))
        if mut is not None and random.random() < mut[1]: # mutation possible and with probability [1]
            return mut[0] # gene from mutation
        return weighted_if(0.5, self, other) # random gene if not mutated

In [ ]:
class BeeGender(Enum):
    DRONE = auto()
    PRINCESS = auto()
    QUEEN = auto()

class BeeSpecies(Enum):
    FOREST      = auto()
    MEADOWS     = auto()
    COMMON      = auto()
    CULTIVATED  = auto()
    MAJESTIC    = auto()
    NOBLE       = auto()
    IMPERIAL    = auto()
    DILIGENT    = auto()
    UNWEARY     = auto()
    INDUSTRIOUS = auto()
    # COMMON    = auto()
    # COMMON    = auto()
    # COMMON    = auto()
    # COMMON    = auto()

bs = BeeSpecies
assert len(BeeSpecies) == 10, 'Dont forget to update basic species'
basic_species = [
    bs.FOREST,
    bs.MEADOWS
]
    
local = {
    BeeGender.DRONE        : 'Drone',
    BeeGender.PRINCESS     : 'Princess',
    BeeGender.QUEEN        : 'Queen',
    BeeSpecies.FOREST      : 'FOREST',
    BeeSpecies.MEADOWS     : 'MEADOWS',    
    BeeSpecies.COMMON      : 'COMMON',     
    BeeSpecies.CULTIVATED  : 'CULTIVATED', 
    BeeSpecies.MAJESTIC    : 'MAJESTIC',   
    BeeSpecies.NOBLE       : 'NOBLE',      
    BeeSpecies.IMPERIAL    : 'IMPERIAL',   
    BeeSpecies.DILIGENT    : 'DILIGENT',   
    BeeSpecies.UNWEARY     : 'UNWEARY',    
    BeeSpecies.INDUSTRIOUS : 'INDUSTRIOUS',
    2: '2'
}
    

mutations = {
    (Allele(bs.FOREST,     True), Allele(bs.MEADOWS, True)): (Allele(bs.COMMON,     True), 0.5),
    (Allele(bs.FOREST,     True), Allele(bs.COMMON,  True)): (Allele(bs.CULTIVATED, True), 1),
    (Allele(bs.COMMON,     True), Allele(bs.MEADOWS, True)): (Allele(bs.CULTIVATED, True), 1),
    (Allele(bs.CULTIVATED, True), Allele(bs.COMMON,  True)): (Allele(bs.MAJESTIC,   True), 1),
}
mutations.update({ (key[1], key[0]) : mutations[key] for key in mutations})
del bs

In [ ]:
Gene = Tuple[Allele, Allele]

@dataclass
class Genes:
    species: Gene
    fertility: Gene
    
    def crossingover(self, genes2):
        l = []
        genes1_dict = vars(self)
        genes2_dict = vars(genes2)
        for key1, key2 in zip(genes1_dict, genes2_dict):
            assert key1 == key2, 'fields should be equal...'
            g1 = genes1_dict[key1]
            g2 = genes2_dict[key2]
            
            allele11 = weighted_if(0.5, *g1)
            allele12 = weighted_if(0.5, *g2)
            allele21 = weighted_if(0.5, *g1)
            allele22 = weighted_if(0.5, *g2)
            
            allele1 = allele11.mutate(allele12)
            allele2 = allele21.mutate(allele22)
            if not allele1.dominant and allele2.dominant:
                l.append((allele2, allele1))
            else:
                l.append((allele1, allele2))
        return Genes(*l)
    
    @staticmethod
    def sample(basic=True):
        if basic:
            species = Allele(random.sample(basic_species, 1)[0], True)
            fertility = Allele(2, True)
            return Genes(
                (species, species),
                (fertility, fertility)
            )

In [ ]:
class Bee:
    def __init__(self, gender: BeeGender, genes: Genes, inspected: bool = False):
        self.gender = gender
        self.genes = genes
        self.inspected = inspected
    
    def __str__(self):
        res = []
        if not self.inspected:
            res.append(self.small_str())
            return '\n'.join(res)
        res.append(local[self.gender])
        genes = vars(self.genes)
        for key in genes:
            res.append(key+' : '+local[genes[key][0].value]+' dom: '+str(genes[key][0].dominant)+' '+local[genes[key][1].value]+' dom: '+str(genes[key][0].dominant))
        return '\n'.join(res)
    
    def small_str(self):
        return local[self.genes.species[0].value] + ' ' + local[self.gender]
    
    def render(self):
        if not self.inspected:
            print(self)
            return
        print(local[self.gender])
        genes = vars(self.genes)
        for key in genes:
            print(key, ':', local[genes[key][0].value], 'dom:', genes[key][0].dominant, local[genes[key][1].value], 'dom:', genes[key][0].dominant)

class Queen(Bee):
    def __init__(self, g1, g2, lifespan, inspected: bool = False):
        self.g1 = g1
        self.g2 = g2
        g = self.g1.crossingover(self.g2)
        super().__init__(BeeGender.QUEEN, g, inspected)
        self.lifespan = lifespan
        self.remaining_lifespan = lifespan
    
    def small_str(self):
        return super().small_str() + ', rem: ' + str(self.remaining_lifespan)
    
    def die(self):
        return [Princess(self.genes)] + [Drone(self.g1.crossingover(self.g2)) for i in range(self.genes.fertility[0].value)]
    
    def render(self):
        super().render()
        print('ticks remaining:', self.remaining_lifespan)
    
class Princess(Bee):
    def __init__(self, genes, inspected: bool = False):
        super().__init__(BeeGender.PRINCESS, genes, inspected)
    
    def mate(self, other: 'Drone') -> Queen:
        if other.gender is not BeeGender.DRONE:
            raise TypeError('Princesses can only mate drones')
        return Queen(self.genes, other.genes, 3)

class Drone(Bee):
    def __init__(self, genes, inspected: bool = False):
        super().__init__(BeeGender.DRONE, genes, inspected)

In [ ]:
pf = Princess(Genes(
        (Allele(BeeSpecies.FOREST, True), Allele(BeeSpecies.FOREST, True)),
        (Allele(2, True), Allele(2, True)),
    ))
dm = Drone(Genes(
        (Allele(BeeSpecies.MEADOWS, True), Allele(BeeSpecies.MEADOWS, True)),
        (Allele(2, True), Allele(2, True)),
    ))
qc = pf.mate(dm)
pc, dc, dc2 = qc.die()
qcu = pc.mate(dm)
pcu, dcu, dcu2 = qcu.die()
print(pcu.mate(dc))

In [ ]:
import threading
import time

import ipywidgets as widgets
class Inventory:
    def __init__(self, capacity=None, print_=None):
        self.capacity = capacity or 100
        self.storage = [None] * self.capacity
        if print_ is None:
            self.print = print
        else:
            self.print = print_
        
    def __setitem__(self, key, value):
        if self.storage[key] is None:
            self.storage[key] = value
        else:
            raise IndexError('The slot is not empty')
    
    def __getitem__(self, key):
        return self.storage[key]
    
    def __str__(self):
        res = ['------ INV ------']
        
        empty = True
        for index, bee in enumerate(self.storage):
            if bee is not None:
                empty = False
                res.append(str(index) + ' ' + bee.small_str())
        if empty:
            res.append('Empty...')
        
        return '\n'.join(res)
    
    def pop(self, index):
        bee = self.storage[index]
        self.storage[index] = None
        return bee
    
    @property
    def empty_slots(self):
        return len([el is not None for el in self.storage])
    
    def swap(self, i1, i2):
        self.storage[i1], self.storage[i2] = self.storage[i2], self.storage[i1]
        
    def mate(self, i1, i2):
        if i1 == i2:
            raise ValueError("Can't mate a bee with itself")
        self.storage[i1] = self.storage[i1].mate(self.storage[i2])
        self.storage[i2] = None
        
    def render(self):
        self.print(self)
                
    def place_bees(self, offspring, parent_index=None):
        if parent_index is not None:
            self.storage[parent_index] = None
        index = 0
        for bee in offspring:
            while index < self.capacity:
                try:
                    self.__setitem__(index, bee)
                    index += 1
                    break
                except IndexError:
                    index += 1
                    continue
            else:
                self.print('Beware that', bee, 'was thrown out...')
    

In [ ]:
inv = Inventory(100)
inv[0] = pf
inv[1] = dm
inv.render()
inv.mate(0, 1)
inv.render()
inv.place_bees(inv[0].die(), 0)
inv.render()

In [ ]:
class Apiary:
    def __init__(self, add_resources):
        self.inv = Inventory(7)
        self.princess = None
        self.drone = None
        self.add_resources = add_resources
        
    def __str__(self):
        res = ['------ APIARY ------']
        print(self.princess)
        print(self.drone)
        if self.princess is not None:
            res.append('Princess: ' + self.princess.small_str())
        else:
            res.append('Princess slot is empty')
        if self.drone is not None:
            res.append('Drone: ' + self.drone.small_str())
        else:
            res.append('Drone slot is empty')
            
        res.append(str(self.inv))
        return '\n'.join(res)
    
    def put_princess(self, bee):
        if not isinstance(bee, Princess):
            raise TypeError('Bee should be a Princess')
        self.princess = bee
        self.try_breed()
    
    def put_drone(self, bee):
        if not isinstance(bee, Drone):
            raise TypeError('Bee should be a Drone')
        self.drone = bee
        self.try_breed()
    
    def try_breed(self):
        if self.princess is not None and self.drone is not None:
            self.princess = self.princess.mate(self.drone)
        self.drone = None
    
    def update(self):
        if self.princess.gender == BeeGender.QUEEN:
            if self.princess.remaining_lifespan == 0:
                bees = self.princess.die()
                if len(bees) <= self.inv.empty_slots:
                    self.inv.place_bees(bees)
                    self.princess = None
            else:
                self.princess.remaining_lifespan -= 1
                self.add_resources(honey=1)
            

In [ ]:
h = 0
def add(honey):
    global h
    h += honey
api = Apiary(add_resources=add)
api.put_princess(pf)
print(api)
api.put_drone(dm)
print(api)
api.update()
print(api)
api.update()
api.update()
print(api)
api.update()
print(api)
print(h)

In [ ]:
del inv, pf, dm, qc, pc, dc, dc2, qcu, pcu, dcu, dcu2

In [ ]:
def forage():
    genes = Genes.sample()
    return Princess(genes), Drone(genes)

In [ ]:
class CLI:
    def __init__(self):
        self.command_lock = threading.Lock()
        self.command_event = threading.Event()
        self.state_changed = threading.Event()
        self.state_changed.set()
        self.exit_event = threading.Event()
       
        def f():
            self.print('honey:', self.honey)
            self.print(self.inv, flush=True)
        self.render_func = f
        
        self.out = widgets.Output()
        self.command_out = widgets.Output()
        self.text = widgets.Text()
        self.text_memory = []
        self.text_counter = -1
        self.button_left = widgets.Button(
            icon='arrow-left'
        )
        self.button_right = widgets.Button(
            icon='arrow-right'
        )
        self.text.on_submit(self.register_command)
        self.button_left.on_click(self.text_previous)
        self.button_right.on_click(self.text_next)
        display(widgets.VBox([self.out, self.command_out, widgets.HBox([self.text, self.button_left, self.button_right])]))
        
        self.honey = 0
        self.inv = Inventory(100)
        
        self.print_buffer = []
        self.print_buffer_size = 0
        
        self.inner_state_thread = threading.Thread(target=self.update_state)
        self.inner_state_thread.start()
        self.render_thread = threading.Thread(target=self.render)
        self.render_thread.start()
        
        
    def print(self, *strings, sep=' ', end='\n', flush=False, append=False, out=None):
        if out is None:
            out = self.out
        thing = sep.join(map(str, strings)) + end
        
        if append:
            out.append_stdout(thing)
            return
        
        self.print_buffer.append(thing)
        self.print_buffer_size += len(thing)
        if flush:
            out.outputs = ({'output_type': 'stream',
              'name': 'stdout',
              'text': ''.join(self.print_buffer)},) # very evil hack, it will clear the screen every time you flush (and without flush, you won't even see anything)
            self.print_buffer = []
            self.print_buffer_size = 0
        
    def render(self):
        while True:
            if self.exit_event.is_set():
                self.print(flush=True)
                break
            
            if self.state_changed.is_set():
                self.render_func()
                self.state_changed.clear()
    
    def register_command(self, b):
        self.print(self.text.value, out=self.command_out, flush=True)
        if self.exit_event.is_set():
            return
        command, *params = self.text.value.split()
        if command in ['exit', 'q']:
            self.exit_event.set()
            self.print('Exiting...')
        elif command == 'inv':
            if len(params) == 0:
                def f():
                    self.print('honey:', self.honey)
                    self.print(self.inv, flush=True)
                self.render_func = f
            else:
                slot = int(params[0])
                def f():
                    self.print('honey:', self.honey)
                    bee = self.inv[slot]
                    if bee is not None:
                        self.print(bee, flush=True)
                    else:
                        self.print('Slot empty', flush=True)
                self.render_func = f
            self.state_changed.set()
        elif command == 'mate':
            try:
                a, b = map(int, params)
            except ValueError as e:
                print(e)
            else:
                try:
                    self.inv.mate(a, b)
                except (TypeError, ValueError) as e:
                    print(e)
                else:
                    self.print('after mate', out=self.command_out, flush=True)
                    self.state_changed.set()
        elif command == 'swap':
            self.inv.swap(*map(int, params))
            self.state_changed.set()
        elif command == 'forage':
            res = forage()
            self.inv.place_bees(res)
            self.state_changed.set()
        elif command == 'inspect':
            try:
                bee = self.inv[int(params[0])]
                if bee is not None and self.honey >= 10 and not bee.inspected:
                    bee.inspected = True
                    self.honey -= 10
                    self.state_changed.set()
            except (IndexError, ValueError) as e:
                print(e)
        if len(self.text_memory) == 0 or self.text.value != self.text_memory[-1]:
            self.text_memory.append(self.text.value)
        self.text_counter = len(self.text_memory)
        self.text.value = ''

    def text_next(self, b):
        self.text_counter = min(self.text_counter + 1, len(self.text_memory))
        if self.text_counter == len(self.text_memory):
            self.text.value = ''
        else:
            self.text.value = self.text_memory[self.text_counter]
    
    def text_previous(self, b):
        self.text_counter = max(self.text_counter - 1, 0)
        if self.text_counter != 0 or len(self.text_memory) != 0:
            self.text.value = self.text_memory[self.text_counter]
        
    def add_resourses(self, honey=0):
        self.honey += honey
        
    def update_state(self):
        while True:
            time.sleep(1)
            if self.exit_event.is_set():
                break
            
            for index, bee in enumerate(self.inv):
                if bee is not None and bee.gender == BeeGender.QUEEN:
                    if bee.remaining_lifespan == 0:
                        self.inv.place_bees(bee.die(), index)
                    else:
                        bee.remaining_lifespan -= 1
                        self.honey += 1
                    self.state_changed.set()
            
            

In [ ]:
cli = CLI()

In [ ]:
1/0

In [ ]:
from IPython.display import display
event = threading.Event()
out = widgets.Output()
text = widgets.Text()
button = widgets.Button(icon='check')

def print_my(b):
    if text.value == 'disarm':
        other_function(out)
        event.set()
    else:
        out.append_stdout('not disarm\n')

button.on_click(print_my)
    
display(widgets.VBox([out, text, button]))

def background(out):
    while True:
        if event.is_set():
            break
        out.outputs = tuple()
        out.append_stdout('disarm me by typing disarm\n')


def other_function(out):
        out.append_stdout('You disarmed me! Dying now.\n')

# now threading1 runs regardless of user input
threading1 = threading.Thread(target=background, args=(out,))
threading1.start()